## imports


In [ ]:
!pip install -r requirements.txt
import os
os.environ["PYTHONHASHSEED"] = str(0)
from copy import deepcopy
from sklearn.metrics import mean_absolute_error, mean_squared_error
import imgaug
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageOps
import cv2
import random
import glob
import seaborn as sns
import wandb
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.distributed import DistributedSampler
import albumentations as A
from albumentations.pytorch import ToTensorV2
import pytorch_lightning as pl

from tqdm.auto import tqdm
import gc
import segmentation_models_pytorch as smp
from segmentation_models_pytorch.utils.losses import JaccardLoss
from segmentation_models_pytorch.utils.metrics import IoU

import warnings
import networkx as nx
import os
from sklearn.metrics import precision_score, recall_score, f1_score
warnings.filterwarnings("ignore")
from datetime import datetime
import pandas as pd
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
from pathlib import Path
from torchvision.utils import save_image
from torchvision.transforms import ToTensor
from tqdm.auto import trange
from scipy.stats import f_oneway, kruskal


In [ ]:
from inference.py import *
from stability_metrics.py import *
set_seed(0)

In [13]:
mean = [0.253317  , 0.26740879, 0.23025433]
std  = [0.15158384, 0.14880167, 0.14123519]

test_augs = A.Compose([
    # A.Flip(p=0.5),
    # A.Rotate(border_mode=0, p=0.5),
    A.RandomScale(scale_limit=0.2, p=0.5),
    A.RandomBrightness(limit=0.5, p=0.8),
    A.RandomContrast(limit=0.5, p=0.8),
    A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=20, val_shift_limit=20, p=0.8),
    #A.Normalize(mean=mean, std=std, max_pixel_value=255.0, always_apply=True),
    A.PadIfNeeded(512,512, border_mode=0),    
    A.CenterCrop(512,512, always_apply=True),
    ToTensorV2()
])

val_augs = A.Compose([
    A.Normalize(mean=mean, std=std, max_pixel_value=255.0, p=1.0),
    ToTensorV2(p=1.0)
])


contrast = np.load('E:/diplom_dataset/samples_cheater/test/contrast1.npy')
brightness = np.load('E:/diplom_dataset/samples_cheater/test/brightness1.npy')
hue = np.load('E:/diplom_dataset/samples_cheater/test/hue.npy')
angles = np.load('E:/diplom_dataset/samples_cheater/test/angles.npy')
scale_ = np.load('E:/diplom_dataset/samples_cheater/test/scale_.npy')

# choose distortions
torchvision_transform = {
                        # transforms.functional.adjust_contrast:contrast,
                        # transforms.functional.adjust_brightness:brightness,
                        #transforms.functional.adjust_hue:hue
                        transforms.functional.rotate:angles,
                        scale:scale_
                        }

batch_siZe = 10 #50
orig_dataloader = get_test_dataloader('E:/diplom_dataset/samples_cheater/test/', val_augs, batch_siZe, num_workers=0)

## 1. Метрика зависящая от ground truth (Сравниваем значение метрик качества)

In [ ]:
#model_name = 'unet34_20_8_0_5e-05_w_less_color_augs'
model_names = ['unet34_20_8_0_5e-05', 'unet34_20_8_0_5e-05_w_strct_augs']

for model_name in model_names:
    model = ModelPl.load_from_checkpoint('unet_resnet34/'+ model_name +'.ckpt')
    Predictor = SimplePredictor(model.cuda(), device=device, 
                                sample_size=(512,512), 
                                #sample_size=(1024,1024), 
                                pad_size=64)
    scores = calculate_metrics(Predictor, 'E:/diplom_dataset/samples_cheater/test', 
                            augs=val_augs, 
                            augs_dict = torchvision_transform,
                            threshold=0.5)

    df = pd.DataFrame(pd.DataFrame(scores))
    print(df)
    df.to_csv('metrics/files/augs_struct_img_'+ model_name +'.csv', index=False) 
    #df.to_csv('metrics/files/orig_img_'+ model_name +'.csv', index=False) 

In [ ]:
model_name_1 = 'unet34_20_8_0_5e-05'         
#model_name_2 =  'unet34_20_8_0_5e-05_w_color_augs'   
#model_name_2 = 'unet34_20_8_0_5e-05_w_less_color_augs'
model_name_2 = 'unet34_20_8_0_5e-05_w_strct_augs'
metric_name = 'IoU'

print(superwised_metrics(model_name_1,metric_name))
print(superwised_metrics(model_name_2,metric_name))

## 2. Метрики не зависящие от ground truth

Сравниваем результаты моделей

In [ ]:
# plot image with augs
plt.imshow(scale(next(iter(orig_dataloader))['mask'], 0.5)[0].T)

In [ ]:
# all augs
iou = IoU() 
mae_f = torch.nn.L1Loss(reduction='none')
mse_f = torch.nn.MSELoss(reduction='none')
@torch.no_grad()

# ['unet34_20_8_0_5e-05', 'unet34_20_8_0_5e-05_w_less_color_augs',
model_names = ['unet34_20_8_0_5e-05', 'unet34_20_8_0_5e-05_w_strct_augs']  


torchvision_transform_color = {transforms.functional.adjust_contrast:contrast,
                                transforms.functional.adjust_brightness:brightness,
                                #transforms.functional.adjust_hue:hue
                                }
torchvision_transform_struct = {transforms.functional.rotate:angles,
                        scale:scale_}

for model_name in model_names:
    model = ModelPl.load_from_checkpoint('unet_resnet34/'+ model_name +'.ckpt').cuda()
    print(model_name, loader_metircs(model, orig_dataloader=orig_dataloader, augs_dict = torchvision_transform_color, threshold=0.501))

In [ ]:
# imshow predict
model = ModelPl.load_from_checkpoint('unet_resnet34/unet34_20_8_0_5e-05.ckpt').cuda()
img = next(iter(orig_dataloader))['img']
img_augs = transforms.functional.adjust_contrast(img,contrast[0]).cuda()
pred_augs = model(img_augs)
plt.imshow(pred_augs[0].T.detach().cpu()) 
plt.show()

## 3.1. Test on prod models (buildings)

In [ ]:
from typing import Final

architectures: Final[dict] = {'Unet': smp.Unet,
                              'Unet++': smp.UnetPlusPlus,
                              'MAnet': smp.MAnet,
                              'DeepLabV3+': smp.DeepLabV3Plus}

encoders: Final[dict] = {'mit-b2': 'mit_b2',
                         'mit-b3': 'mit_b3',
                         'efficientnet-b1': 'efficientnet-b1',
                         'efficientnet-b2': 'efficientnet-b2',
                         'efficientnet-b3': 'efficientnet-b3',
                         'efficientnet-b4': 'efficientnet-b4',
                         'efficientnet-b5': 'efficientnet-b5',
                         'timm-res2net50-26w-4s': 'timm-res2net50_26w_4s'}

models_root = 'D:\diploma\cv-corruption-research\models'
model_names = ['DeepLabV3+_efficientnet-b4' , 'MAnet_efficientnet-b4', 'Unet_mit-b2', 
                'Unet++_efficientnet-b5']

model_name = model_names[0]

segm_arch = model_name.split('_')[0]
encoder = model_name.split('_')[1]

model = architectures[segm_arch](in_channels = 3, classes=4, 
                                 encoder_name = encoders[encoder],
                                 encoder_weights=None,
                                 activation = None)

model.load_state_dict(torch.load(os.path.join(models_root, f'{model_name}.pth')))
model.cuda()
model.eval();
print(model)
#next(model.parameters()).is_cuda

In [ ]:
loader_metircs(model, orig_dataloader=orig_dataloader, augs_dict = torchvision_transform, threshold=0.501)

open images and count metric

In [ ]:
orig_dataloader = get_test_dataloader('E:/diplom_dataset/samples_cheater/test/', val_augs, 1, num_workers=0)
batch = next(iter(orig_dataloader))

i, j  = 0, 0
img_orig = batch['img'].cuda()

img_augs = img_orig.clone().detach()


for k in range(len(img_orig)):
    #print(k)
    for function, parameters in torchvision_transform.items():
        img_augs[k] = function(img_augs[k], parameters[i])  
        #print('i', i, function, parameters)             
    i+=1


pred_orig = model(img_orig).sigmoid() 
pred_augs = model(img_augs).sigmoid() 

for k in range(len(img_orig)):
    for function, parameters in torchvision_transform.items():
        if function in [transforms.functional.rotate]:
            pred_orig[k] = function(pred_orig[k], parameters[j])
    j+=1

mape_ = mape_f(pred_orig, pred_augs).flatten()

## 3.2. Test on prod models (forest)

In [ ]:
# plot image with augs

model = torch.load('forest\model07.pt')

img = next(iter(orig_dataloader))['img']
img_augs = transforms.functional.adjust_hue(img.int(), contrast[0])

pred_orig = model(img.cuda())
pred_orig = pred_orig.sigmoid()
plt.imshow(pred_orig[0].T.cpu()) 
plt.show()

pred_augs = model(img_augs.cuda())
pred_augs = pred_augs.sigmoid()
plt.imshow(pred_augs[0].T.cpu()) 
plt.show()


In [ ]:
iou = IoU() 
mae_f = torch.nn.L1Loss(reduction='none')
mse_f = torch.nn.MSELoss(reduction='none')
@torch.no_grad()

for model_name in ['forest\model07.pt',
                   'forest\model08.pt',
                   'forest\model09.pt']:
    model = torch.load(model_name)
    print(model_name)
    print(loader_metircs(model, orig_dataloader=orig_dataloader, augs_dict = torchvision_transform, threshold=0.501))

## 4. Appendix

## rotate predict mask for predict image

In [ ]:
save_augs = A.Compose([
    ToTensorV2(p=1.0)
])

img_paths = sorted(glob.glob('E:/diplom_dataset/samples_cheater\\test\images\*.tif'))
mask_paths = sorted(glob.glob('E:/diplom_dataset/samples_cheater\\test\masks\*.tif'))

angles = np.load('E:/diplom_dataset/samples_cheater/test/angles.npy')

for i in trange(len(img_paths)):
    image=Image.open(img_paths[i])
    mask=Image.open(mask_paths[i])

    image = np.array(torchvision.transforms.functional.rotate(image, angles[i]))
    mask = np.array(torchvision.transforms.functional.rotate(mask, angles[i]))[:,:,0]

    trans = save_augs(image=image, mask=mask)
    img, mask = trans['image'], trans['mask']
    save_image(img.float()/255, f'E:/diplom_dataset/samples_cheater/test/images/' + Path(img_paths[i]).name)
    save_image(mask.float(), f'E:/diplom_dataset/samples_cheater/test/masks/' + Path(mask_paths[i]).name)


## save test dataset with augs

In [21]:
img_paths = sorted(glob.glob('E:/diplom_dataset/samples_cheater\\test\images\*.tif'))
mask_paths = sorted(glob.glob('E:/diplom_dataset/samples_cheater\\test\masks\*.tif'))

for i in trange(len(img_paths)):
    trans = test_augs(image=np.array(Image.open(img_paths[i])), mask=np.array(Image.open(mask_paths[i]))[:,:,0])
    img, mask = trans['image'], trans['mask']
    save_image(img.float()/255, f'E:/diplom_dataset/samples_cheater/test_color_augs_l/images/' + Path(img_paths[i]).name)
    save_image(mask.float(), f'E:/diplom_dataset/samples_cheater/test_color_augs_l/masks/' + Path(mask_paths[i]).name)


100%|██████████| 1518/1518 [00:30<00:00, 50.39it/s]
